In [ ]:
# Import the necessary packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import sqlite3
sns.set()

# Importing all the .db files and combining them into a dataframe

In [ ]:
# Create hashtags dataframe
Hashtags = pd.DataFrame()

In [ ]:
# Getting all monthly files containing the information about hashtags
# Connect with the db file and create cursor object to be able to execute queries
year = 2012
month = 1

yearStr = ''
monthStr = ''

while year < 2019:
    
    while month < 13:
        
        # Change the month to corresponding string.
        if month == 1:
            monthStr = '01'
        if month == 2:
            monthStr = '02'    
        if month == 3:
            monthStr = '03'
        if month == 4:
            monthStr = '04'
        if month == 5:
            monthStr = '05'
        if month == 6:
            monthStr = '06'
        if month == 7:
            monthStr = '07'    
        if month == 8:
            monthStr = '08'
        if month == 9:
            monthStr = '09'
        if month == 10:
            monthStr = '10'
        if month == 11:
            monthStr = '11'
        if month == 12:
            monthStr = '12'
        
        yearStr = str(year)
        # Create connection
        connection = sqlite3.connect('/Volumes/Hard drive/Thesis/Data/' + yearStr + '/month_' + yearStr + '_' + monthStr +'.db')
        cursor = connection.cursor()
        try:
            # Read media nl file and store it in the final dataframe.
            media = pd.read_sql_query("SELECT * FROM hashtags_NL", connection)
            Hashtags = Hashtags.append(media)
        except:
            print('skipped')
        # Add one month
        month += 1
        
    # Reset month
    month = 1
    
    # Add one year
    year += 1

# Creating a hashtag dataframe

In [ ]:
Hashtags=Hashtags.rename(columns={'text':'hashtags'})

In [ ]:
HashtagDict = {}

In [ ]:
uniqueItems = list(Hashtags['item_number'].unique())

In [ ]:
# Create a dictionary of hashtags per item (per tweet)
for i in uniqueItems:
    
    # Create a dataframe with only hashtags for this item.
    hashtagsForThisItem = Hashtags[Hashtags['item_number'] == i]
    
    # Creat an empty list to fill with the hashtags.
    hashtagList = []
    
    # Iterate over the created dataframe to fill in the hashtagList.
    for index, row in hashtagsForThisItem.iterrows():
        hashtagList.append(row['hashtags'])
    
    # Update the dictionary with a key value pair.
    HashtagDict[i] = hashtagList

# Create a dataframe from the dictionary
df = pd.DataFrame.from_dict(HashtagDict, orient='index')

# Save it to a .csv
df.to_csv('TwitterHashtags_Processed.csv')

# Import the .csv to change the column name of 'unnamed: 0' to 'item_number'.
df_temp = pd.read_csv('TwitterHashtags_Processed.csv', sep= ',', low_memory = False, lineterminator='\n')

# Rename the column.
df_temp = df_temp.rename(columns={'Unnamed: 0':'item_number'})

# Export the final dataframe to .csv
df_temp.to_csv('TwitterHashtags_Processed.csv', index=False)

# Grouping tweets based on hashtag

In [ ]:
# Import the tourist dataset
Tourists = pd.read_csv('tourists_total.csv', sep= ',', low_memory = False, lineterminator='\n')

In [ ]:
# Filter on language, starting with english, netherlands, spanish, french, german
Tourists = Tourists[Tourists['lang'] == 'de']

In [ ]:
Tourists['lang'].value_counts()

In [ ]:
# Import the hashtag dataset
Hashtags = pd.read_csv('TwitterHashtags_Processed.csv', sep= ',', low_memory = False, lineterminator='\n')

In [ ]:
# Create an LDA dataset with only the necessary information.
LDA_Total = Tourists[['item_number','text','lang']]

# Merge the dataset so that it contains the hashtags per tweet.
LDA_Total = pd.merge(LDA_Total, Hashtags, on='item_number', how='left')

# Store all the hashtags in one string per tweet.
hashtagStringDict = {}

# Iterate over the entire dataframe to fill the dictionary.
for index, row in LDA_Total.iterrows():
    
    # Create a hashtag list.
    hashtagList = []
        
    # Create missing value list.
    missingValueList = list(row.isnull())[3:]
        
    # Iterate over all the hashtag columns and check if it is a missing value.
    for i in range(41):
        if missingValueList[i] == False:
            hashtagList.append(row[str(i)])
        
    # Store all the hashtags in one string
    hashtagString = ', '.join(hashtagList)
    hashtagStringDict[row['item_number']] = hashtagString
    
# Change the dictionary to a dataframe.
HashtagInfo_LDA = pd.DataFrame(hashtagStringDict, index=['hashtags']).T
HashtagInfo_LDA['item_number'] = HashtagInfo_LDA.index   

# Merge the newly created dataframe and drop all unneccessary columns.
LDA_Total = pd.merge(LDA_Total, HashtagInfo_LDA, on='item_number', how='inner')
LDA_Total = LDA_Total[['item_number','text','lang','hashtags']]

In [ ]:
LDA_Total.head()

In [ ]:
# Iterate over the newly created dataframe to create a set of hashtags.

# Empty list of hashtags
hashtagList_Total = []

for index, row in LDA_Total.iterrows():
    
    # Create individual list
    individualList = row['hashtags'].split(',')
    
    # Iterate over the individual list
    for i in individualList:
        
        # Append to the total list
        hashtagList_Total.append(i)

# Transform into a set to retain only unique hashtags, then transform it back to a list to make it iterable.
hashtagList_Total = set(hashtagList_Total)
hashtagList_Total = list(hashtagList_Total)

In [ ]:
# Now, all tweets that contain a certain hashtag need to be aggregated based on that hashtag.

# Empty list, will contain [Hashtag, comma separated item_number, total text]
Hashtags_Aggregated = []

# Iterate over the newly created dataframe.
for index, row in LDA_Total.iterrows():
    
    # For each row, get the hashtags and put them in a list.
    indiv_Tweet_HashtagList = row['hashtags'].split(',')
    
    
    # Create index list to use in the dictionary update after the loop
    indexes = []
    
    # Iterate over that list and see if it is in the total hashtag list (should always be the case)
    for i in indiv_Tweet_HashtagList:
        
        # If the hashtag is in the list, find the index of that hashtag.
        if i in hashtagList_Total:
            
            # Get the index.
            indexes.append(hashtagList_Total.index(i))
    
    # For each index, create new entry in the 2d list.
    for i in indexes:
        
        # Get the value.
        value = hashtagList_Total[i]
        
        # List comprehesion to get the first item of each item.
        hashtagListTemp = [x[0] for x in Hashtags_Aggregated]
        
        # If the value is already in the 2d list.
        if value in hashtagListTemp:
            
            # Get the index.
            indexOfValue = hashtagListTemp.index(value)
            
            # Update the values that belong to this index/hashtag
            Hashtags_Aggregated[indexOfValue][1] += (', ' + str(row['item_number'])) # Add item number.
            Hashtags_Aggregated[indexOfValue][2] += (' ' + row['text']) # Add text.
        
        # Else just add it to the list.
        else:
            Hashtags_Aggregated.append([value, str(row['item_number']), row['text']])
            
    print((index/len(LDA_Total)))


In [ ]:
# Create a dataframe with all the information.
LDA_Hashtag_Total = pd.DataFrame(Hashtags_Aggregated, columns = ['hashtag', 'matched_items', 'text'])  

In [ ]:
LDA_Hashtag_Total

In [ ]:
# Write to csv file.
LDA_Hashtag_Total.to_csv('TwitterHashtags_LDA_DE.csv', index=False)